In [ ]:
import numpy as np
import pandas as pd
import src.utils as utils

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load Configuration File

In [ ]:
config_data = utils.config_load()
config_data

## Data Leakege - 

standarization after splitting data. would overfitting the trained data? 
overfitting; bagus di training (100), jelek di test (msial 30)
kalau data test; bagus di training dan bagus di test (100).

Best: Splitting first, then Standarization
as an example; we should learn from the book and hands-on. following that, we face the exam. 

not the other way around. no back to the future 

because we need our model to:
1. learn from the past;
2. predict the future.

### Kita bakal fokus di training dataaset - akan trade off bias variance


# Load Data

In [ ]:
X_train = utils.pickle_load(config_data['train_set_path'][0])
y_train = utils.pickle_load(config_data['train_set_path'][1])

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)

In [ ]:

##  gabung lagi untuk keperluan EDA

data = pd.concat((X_train, y_train), axis=1)
data.head()


# EDA

### check data type

In [ ]:
# Check datatype
data.dtypes

### check missing value

In [ ]:
# Check missing value
data.isna().sum()

**Summary**

There are no missing values in our data ## yeaaay ##

### Check correlation

In [ ]:
#Plotting a heatmap to visualize the correlation between the variables
plt.figure(figsize = (14,14))
plt.title('Credit Card Transactions features correlation plot (Pearson)')
corr = data.corr()
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,cmap="Greens")
plt.show()

### Check class balance

In [ ]:
print(data['Class'].value_counts())
print('\n')
print(data['Class'].value_counts(normalize=True))

In [ ]:
data["Class"].value_counts().plot(kind = 'pie',explode=[0, 0.1],figsize=(6, 6),autopct='%1.1f%%',shadow=True)
plt.title("Fraudulent and Non-Fraudulent Distribution",fontsize=20)
plt.legend(["Fraud", "Genuine"])
plt.show()

#### Descriptive statistics of time and amount columns

In [ ]:
data.columns

In [ ]:
data[['Time','Amount']].describe()

### Visualize Distribution

#### Column Amount

In [ ]:
# column Amount
col = 'Amount'

plt.figure(figsize=(8,6))
plt.title('Distribution of Transaction Amount', fontsize=14)
sns.distplot(data[col], bins=100)
plt.show()

In [ ]:
# Distribution of amount vs class
fig, axs = plt.subplots(ncols=2,figsize=(16,4))
sns.distplot(data[data['Class'] == 1]['Amount'], bins=100, ax=axs[0])
axs[0].set_title("Distribution of Fraud Transactions")

sns.distplot(data[data['Class'] == 0]['Amount'], bins=100, ax=axs[1])
axs[1].set_title("Distribution of Genuine Transactions")

plt.show()

Fraudulent transaction seems happen for small tickets size, with biggest anomalous amounts being around 2000 dollars.

In [ ]:
# check the amount distribution of fraud
print("Fraud Transaction distribution : \n",data[(data['Class'] == 1)]['Amount'].value_counts().head())
print("\n")
print("Maximum amount of fraud transaction - ",data[(data['Class'] == 1)]['Amount'].max())
print("Minimum amount of fraud transaction - ",data[(data['Class'] == 1)]['Amount'].min())

Summary :

- big amount / outlier does not imply fraud
- most of the fraud data is for small transaction (even zero value transaction)

### Column Time

In [ ]:
# column Time
col = 'Time'

plt.figure(figsize=(8,6))
plt.title('Distribution of Transaction Time', fontsize=14)
sns.distplot(data[col], bins=100)
plt.show()

In [ ]:
# Distribution of amount vs class
fig, axs = plt.subplots(ncols=2,figsize=(16,4))
sns.distplot(data[data['Class'] == 1]['Time'], bins=100, ax=axs[0])
axs[0].set_title("Distribution of Fraud Transactions")

sns.distplot(data[data['Class'] == 0]['Time'], bins=100, ax=axs[1])
axs[1].set_title("Distribution of Genuine Transactions")

plt.show()

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.boxplot(x='Class', y='Time',data = data)

plt.title('Time Distribution for Fraud and Genuine transactions')
plt.show()

Summary :

- The data range between 0 and 175000
- There are 2 peaks, which might indicates the day/night cycle (more transaction is done during the night)

### Rest of the columns

In [ ]:
var = data.columns.values

i = 0
t0 = data.loc[data['Class'] == 0]
t1 = data.loc[data['Class'] == 1]

sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(8,4,figsize=(16,28))

for feature in var:
    i += 1
    plt.subplot(8,4,i)
    sns.kdeplot(t0[feature], bw=0.5,label="Class = 0")
    sns.kdeplot(t1[feature], bw=0.5,label="Class = 1")
    plt.xlabel(feature, fontsize=12)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.legend()
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
importance = model.feature_importances_

# Plot feature importance
plt.figure(figsize=(12, 6))
sns.barplot(x=importance, y=feature_names)
plt.title("Feature Importance")
plt.show()
